In [8]:
import os
import time
import gradio as gr
from typing import TypedDict, Any
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from concurrent.futures import ThreadPoolExecutor, as_completed

# --------------------
# State & Cost Tracking
# --------------------
class AgentState(TypedDict):
    messages: list
    topic: str
    research: str
    synthesized: str
    fact_checked: str
    edited: str
    summary: str
    metrics: dict

PRICES = {
    "gpt-4o-mini": {"input": 0.00015 / 1000, "output": 0.0006 / 1000}
}

def estimate_tokens(text: str) -> int:
    if not text:
        return 0
    return int(len(text.split()) * 1.3)

def _extract_content(resp):
    if resp is None:
        return ""
    if hasattr(resp, "content"):
        return getattr(resp, "content") or ""
    if isinstance(resp, dict) and "content" in resp:
        return resp["content"] or ""
    return str(resp)

# --------------------
# Global LLM (set in setup tab)
# --------------------
llm = None

def call_llm_with_metrics(agent_name, prompt, state):
    global llm
    start = time.time()
    try:
        resp = llm.invoke(prompt)
    except Exception as e:
        resp = None
        print(f"[ERROR] {agent_name} failed:", e)
    end = time.time()
    content = _extract_content(resp)

    in_tokens = estimate_tokens(prompt)
    out_tokens = estimate_tokens(content)
    model = getattr(llm, "model_name", "gpt-4o-mini")
    pricing = PRICES.get(model, PRICES["gpt-4o-mini"])
    cost = in_tokens * pricing["input"] + out_tokens * pricing["output"]

    state.setdefault("metrics", {})[agent_name] = {
        "latency": round(end - start, 2),
        "in_tokens": in_tokens,
        "out_tokens": out_tokens,
        "cost": round(cost, 6),
    }
    return content

# --------------------
# Agents
# --------------------
def ResearchAgent(state: AgentState) -> AgentState:
    prompt = f"""
    You are a Research Agent.

    Task: Gather relevant, high-quality information on the topic below.
    - Cover definitions, background, current trends, and key debates.
    - Include real-world applications or case studies if relevant.
    - Present findings in a structured way (not an essay, not bullet-only).
    - Keep it neutral and fact-rich.

    Topic:
    {state['topic']}
    """
    state["research"] = call_llm_with_metrics("ResearchAgent", prompt, state)
    state["messages"].append("research_done")
    return state

def SynthesizeAgent(state: AgentState) -> AgentState:
    prompt = f"""
    You are a Synthesis Agent.

    Task: Combine the research findings into a **coherent draft article**.
    - Weave information into a clear narrative.
    - Avoid repetition; integrate overlapping points.
    - Maintain a balanced, professional tone.
    - Do not fact-check or edit here — just create a readable draft.

    Research:
    {state['research']}
    """
    state["synthesized"] = call_llm_with_metrics("SynthesizeAgent", prompt, state)
    state["messages"].append("synthesize_done")
    return state


def FactCheckAgent(state: AgentState) -> AgentState:
    prompt = f"""
    You are a Fact-Checking Agent.

    Task: Review the following draft and ensure factual accuracy.
    - Correct inaccuracies *inline*.
    - If information is vague or incomplete, improve it with accurate detail.
    - Do NOT output commentary like "Accurate" or "Inaccurate."
    - Do NOT ask the user questions.
    - Just return a clean, factually correct draft article.

    Draft:
    {state['synthesized'] or state['research']}
    """
    state["fact_checked"] = call_llm_with_metrics("FactCheckAgent", prompt, state)
    state["messages"].append("factcheck_done")
    return state


def EditAgent(state: AgentState) -> AgentState:
    prompt = f"""
    You are an Editing Agent.

    Task: Improve the readability, flow, and clarity of the following draft.
    - Keep factual details intact.
    - Make the text engaging and coherent.
    - Do not shorten excessively; preserve important content.

    Draft:
    {state['fact_checked']}
    """
    state["edited"] = call_llm_with_metrics("EditAgent", prompt, state)
    state["messages"].append("edit_done")
    return state


def SummarizeAgent(state: AgentState) -> AgentState:
    prompt = f"""
    You are a Summarization Agent.

    Task: Summarize the following article in **3–4 concise sentences**.
    Focus on key ideas and implications. Avoid redundancy.

    Article:
    {state['edited']}
    """
    state["summary"] = call_llm_with_metrics("SummarizeAgent", prompt, state)
    state["messages"].append("summarize_done")
    return state


# --------------------
# Graph Builders
# --------------------
def build_single_agent_graph(llm_param):
    workflow = StateGraph(AgentState)
    def single_node(state: AgentState) -> AgentState:
        start = time.time()
        resp = llm_param.invoke(f"Do research, synthesize, fact-check, edit, and summarize:\n{state['topic']}")
        edited = _extract_content(resp)
        summary_resp = llm_param.invoke("Summarize in 3 sentences:\n" + edited)
        summary = _extract_content(summary_resp)
        end = time.time()
        in_tokens = estimate_tokens(state["topic"])
        out_tokens = estimate_tokens(edited + summary)
        model = getattr(llm_param, "model_name", "gpt-4o-mini")
        pricing = PRICES.get(model, PRICES["gpt-4o-mini"])
        cost = in_tokens * pricing["input"] + out_tokens * pricing["output"]
        state["edited"], state["summary"] = edited, summary
        state.setdefault("metrics", {})["SingleAgent"] = {
            "latency": round(end - start, 2),
            "in_tokens": in_tokens,
            "out_tokens": out_tokens,
            "cost": round(cost, 6),
        }
        return state
    workflow.add_node("single", single_node)
    workflow.set_entry_point("single")
    workflow.add_edge("single", END)
    return workflow.compile()

def build_sequential_agent_graph(llm_param):
    wf = StateGraph(AgentState)
    wf.add_node("research", ResearchAgent)
    wf.add_node("synthesize", SynthesizeAgent)
    wf.add_node("fact", FactCheckAgent)
    wf.add_node("edit", EditAgent)
    wf.add_node("summary", SummarizeAgent)
    wf.set_entry_point("research")
    wf.add_edge("research", "synthesize")
    wf.add_edge("synthesize", "fact")
    wf.add_edge("fact", "edit")
    wf.add_edge("edit", "summary")
    wf.add_edge("summary", END)
    return wf.compile()

def build_parallel_agent_graph(llm_param):
    wf = StateGraph(AgentState)
    def parallel_stage(state: AgentState) -> AgentState:
        synth_state, fact_state = state.copy(), state.copy()
        with ThreadPoolExecutor(max_workers=2) as ex:
            futures = {
                ex.submit(SynthesizeAgent, synth_state): "synth",
                ex.submit(FactCheckAgent, fact_state): "fact"
            }
            for fut in as_completed(futures):
                res = fut.result()
                if futures[fut] == "synth":
                    state["synthesized"] = res["synthesized"]
                else:
                    state["fact_checked"] = res["fact_checked"]
        return state
    wf.add_node("research", ResearchAgent)
    wf.add_node("parallel", parallel_stage)
    wf.add_node("edit", EditAgent)
    wf.add_node("summary", SummarizeAgent)
    wf.set_entry_point("research")
    wf.add_edge("research", "parallel")
    wf.add_edge("parallel", "edit")
    wf.add_edge("edit", "summary")
    wf.add_edge("summary", END)
    return wf.compile()

# --------------------
# Runner
# --------------------
def run_workflow(graph, topic: str) -> dict:
    init_state = {"messages": [], "topic": topic, "research": "", "synthesized": "", "fact_checked": "", "edited": "", "summary": "", "metrics": {}}
    return graph.invoke(init_state)

# --------------------
# Setup
# --------------------
def setup_api_key(api_key: str):
    global llm
    if not api_key.strip():
        return "❌ No API key entered"
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=api_key.strip())
    return "✅ API key set successfully"

# --------------------
# Content + Metrics
# --------------------
def generate_with_graph(graph, topic: str):
    result = run_workflow(graph, topic)
    content = result.get("edited") or result.get("summary") or "(no content)"
    metrics = result.get("metrics", {})
    md = "| Agent | Latency | In | Out | Cost |\n|-------|---------|----|-----|------|\n"
    for k, v in metrics.items():
        md += f"| {k} | {v['latency']}s | {v['in_tokens']} | {v['out_tokens']} | ${v['cost']:.6f} |\n"
    return content, md

# --------------------
# Evaluation (LLM Judge)
# --------------------
def evaluate_outputs(single_out, seq_out, par_out):
    global llm
    judge_prompt = f"""You are a judge. Evaluate three outputs on accuracy, coherence, and conciseness.
    
    --- Single Agent ---
    {single_out}
    
    --- Sequential Agents ---
    {seq_out}
    
    --- Parallel Agents ---
    {par_out}
    
    Give scores (1-10) for each and a short justification."""
    resp = llm.invoke(judge_prompt)
    return _extract_content(resp)

# --------------------
# Gradio UI
# --------------------
with gr.Blocks(title="Multi-Agent Pipeline") as app:
    gr.Markdown("# 🔄 Multi-Agent Content Pipeline")

    with gr.Tab("Setup"):
        api_key_box = gr.Textbox(label="Enter OpenAI API Key", type="password")
        setup_btn = gr.Button("Set API Key")
        setup_status = gr.Markdown()
        setup_btn.click(setup_api_key, api_key_box, setup_status)

    with gr.Tab("Single Agent"):
        topic1 = gr.Textbox(label="Topic")
        out1, metrics1 = gr.Markdown(), gr.Markdown()
        gen1 = gr.Button("Generate (Single)")
        gen1.click(lambda t: generate_with_graph(build_single_agent_graph(llm), t), topic1, [out1, metrics1])

    with gr.Tab("Sequential Agents"):
        topic2 = gr.Textbox(label="Topic")
        out2, metrics2 = gr.Markdown(), gr.Markdown()
        gen2 = gr.Button("Generate (Sequential)")
        gen2.click(lambda t: generate_with_graph(build_sequential_agent_graph(llm), t), topic2, [out2, metrics2])

    with gr.Tab("Parallel Agents"):
        topic3 = gr.Textbox(label="Topic")
        out3, metrics3 = gr.Markdown(), gr.Markdown()
        gen3 = gr.Button("Generate (Parallel)")
        gen3.click(lambda t: generate_with_graph(build_parallel_agent_graph(llm), t), topic3, [out3, metrics3])

    with gr.Tab("Evaluation Results"):
        eval_btn = gr.Button("Evaluate All")
        eval_out = gr.Markdown()
        eval_btn.click(evaluate_outputs, [out1, out2, out3], eval_out)

if __name__ == "__main__":
    app.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
